Get weather data for the Dataset

In [24]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

In [25]:
# Add the directory containing the module to the system path
module_path = os.path.abspath(os.path.join('C:\\Users\\samue\\OneDrive\\AIML\\HS2024\\Data Sicence Projekt\\HSLU_DSPRO1_TrafficStatus\\data'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the module
import data_engineering_library as dlib

In [26]:
data_London_UTD19 = dlib.load_data(path=r"C:\Users\samue\OneDrive\AIML\HS2024\Data Sicence Projekt\Data\London_UTD19.csv", nrows=1000000)
data_London_UTD19.head()

,day,interval,detid,flow,occ,error,city,speed
0,2015-05-16,0,EAST_N04/161x1,144.0,0.028333,0.0,london,NaN
1,2015-05-16,300,EAST_N04/161x1,204.0,0.040000,0.0,london,NaN
2,2015-05-16,600,EAST_N04/161x1,216.0,0.039167,0.0,london,NaN
3,2015-05-16,900,EAST_N04/161x1,168.0,0.029167,0.0,london,NaN
4,2015-05-16,1200,EAST_N04/161x1,144.0,0.024167,0.0,london,NaN


In [27]:
unique_days = data_London_UTD19['day'].unique()
unique_days = pd.to_datetime(unique_days)
unique_days

DatetimeIndex(['2015-05-16', '2015-05-15', '2015-05-17', '2015-05-18',
               '2015-05-19', '2015-05-20', '2015-05-21', '2015-05-22',
               '2015-05-23', '2015-10-01', '2015-09-25', '2015-09-26',
               '2015-09-27', '2015-09-28', '2015-09-29', '2015-09-30',
               '2016-05-16', '2016-05-17', '2016-05-18', '2016-05-19',
               '2016-05-20', '2016-05-21', '2016-05-22'],
              dtype='datetime64[ns]', freq=None)

In [28]:
# This is the example from the Open-Meteo API documentation
# import openmeteo_requests

# import requests_cache
# import pandas as pd
# from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
# cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
# retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
# openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
# url = "https://archive-api.open-meteo.com/v1/archive"
# params = {
# 	"latitude": 51.50,
# 	"longitude": -0.12,
# 	"start_date": "2015-10-14",
# 	"end_date": "2015-10-15",
# 	"hourly": "rain"
# }
# responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
# response = responses[0]
# print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
# print(f"Elevation {response.Elevation()} m asl")
# print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
# print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
# hourly = response.Hourly()
# hourly_rain = hourly.Variables(0).ValuesAsNumpy()

# hourly_data = {"date": pd.date_range(
# 	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
# 	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
# 	freq = pd.Timedelta(seconds = hourly.Interval()),
# 	inclusive = "left"
# )}
# hourly_data["rain"] = hourly_rain

# hourly_dataframe = pd.DataFrame(data = hourly_data)
# print(hourly_dataframe)

In [29]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
url = "https://archive-api.open-meteo.com/v1/archive"

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each unique day value
for day in unique_days:
    # Convert day to string format for API call
    day_str = day.strftime('%Y-%m-%d')
    
    # Set the parameters for the API call
    params = {
        "latitude": 51.50,
        "longitude": -0.12,
        "start_date": day_str,
        "end_date": day_str,
        "hourly": "rain"
    }
    
    # Call the API
    responses = openmeteo.weather_api(url, params=params)
    
    # Process the response
    response = responses[0]
    
    # Process hourly data
    hourly = response.Hourly()
    hourly_rain = hourly.Variables(0).ValuesAsNumpy()
    
    hourly_data = {"date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )}
    hourly_data["rain"] = hourly_rain
    
    hourly_dataframe = pd.DataFrame(data=hourly_data)
    
    # Append the DataFrame to the list
    dataframes.append(hourly_dataframe)

# Combine all DataFrames into a single DataFrame
dataframe_response_rain = pd.concat(dataframes, ignore_index=True)

dataframe_response_rain.head()

,date,rain
0,2015-05-16 00:00:00+00:00,0.0
1,2015-05-16 01:00:00+00:00,0.0
2,2015-05-16 02:00:00+00:00,0.0
3,2015-05-16 03:00:00+00:00,0.0
4,2015-05-16 04:00:00+00:00,0.0


In [30]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
url = "https://archive-api.open-meteo.com/v1/archive"

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each unique day value
for day in unique_days:
    # Convert day to string format for API call
    day_str = day.strftime('%Y-%m-%d')
    
    # Set the parameters for the API call
    params = {
        "latitude": 51.50,
        "longitude": -0.12,
        "start_date": day_str,
        "end_date": day_str,
        "hourly": "cloud_cover"
    }
    
    # Call the API
    responses = openmeteo.weather_api(url, params=params)
    
    # Process the response
    response = responses[0]
    
    # Process hourly data
    hourly = response.Hourly()
    hourly_cloud_cover = hourly.Variables(0).ValuesAsNumpy()
    
    hourly_data = {"date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )}
    hourly_data["cloud_cover"] = hourly_cloud_cover
    
    hourly_dataframe = pd.DataFrame(data=hourly_data)
    
    # Append the DataFrame to the list
    dataframes.append(hourly_dataframe)

# Combine all DataFrames into a single DataFrame
dataframe_response_cloud_cover = pd.concat(dataframes, ignore_index=True)

dataframe_response_cloud_cover.head()

,date,cloud_cover
0,2015-05-16 00:00:00+00:00,87.300003
1,2015-05-16 01:00:00+00:00,74.400002
2,2015-05-16 02:00:00+00:00,100.000000
3,2015-05-16 03:00:00+00:00,100.000000
4,2015-05-16 04:00:00+00:00,100.000000


In [31]:
dataframe_rain = dataframe_response_rain
dataframe_cloud_cover = dataframe_response_cloud_cover
dataframe_rain['date'] = pd.to_datetime(dataframe_rain['date'])
dataframe_cloud_cover['date'] = pd.to_datetime(dataframe_cloud_cover['date'])

dataframe_weather = pd.merge(dataframe_rain, dataframe_cloud_cover, on=[ 'date'], how='inner')
dataframe_weather.head()

,date,rain,cloud_cover
0,2015-05-16 00:00:00+00:00,0.0,87.300003
1,2015-05-16 01:00:00+00:00,0.0,74.400002
2,2015-05-16 02:00:00+00:00,0.0,100.000000
3,2015-05-16 03:00:00+00:00,0.0,100.000000
4,2015-05-16 04:00:00+00:00,0.0,100.000000


In [32]:

# Extract date as string and time interval
dataframe_weather['date_string'] = dataframe_weather['date'].dt.strftime('%Y-%m-%d')
#dataframe_rain['interval'] = dataframe_rain['date'].dt.strftime('%H:%M:%S')
dataframe_weather['interval'] = dataframe_weather['date'].dt.hour * 3600 + dataframe_weather['date'].dt.minute * 60 + dataframe_weather['date'].dt.second

dataframe_weather.drop(columns=['date'], inplace=True)

dataframe_weather.head()

,rain,cloud_cover,date_string,interval
0,0.0,87.300003,2015-05-16,0
1,0.0,74.400002,2015-05-16,3600
2,0.0,100.000000,2015-05-16,7200
3,0.0,100.000000,2015-05-16,10800
4,0.0,100.000000,2015-05-16,14400


In [33]:
dataframe_weather.to_csv(r"C:\Users\samue\OneDrive\AIML\HS2024\Data Sicence Projekt\Data\London_UTD19_weather.csv", index=False)